<a href="https://colab.research.google.com/github/shiyu-cheng/6895_Visual_Sentiment_Prediction/blob/master/image_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install flickrapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 6.4 MB/s eta 0:00:00


In [3]:
from flickrapi import FlickrAPI

FLICKR_PUBLIC = 'f8b9cd454275fca40682a0e602c67073'
FLICKR_SECRET = 'cab676a88194ebae'

flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')
extras='url_sq,url_t,url_s,url_q,url_m,url_n,url_z,url_c,url_l,url_o'
cats = flickr.photos.search(text='kitten', per_page=5, extras=extras)
photos = cats['photos']
from pprint import pprint
pprint(photos)

{'page': 1,
 'pages': 12974,
 'perpage': 5,
 'photo': [{'farm': 66,
            'height_c': 571,
            'height_l': 731,
            'height_m': 357,
            'height_n': 229,
            'height_o': 1500,
            'height_q': 150,
            'height_s': 171,
            'height_sq': 75,
            'height_t': 71,
            'height_z': 457,
            'id': '52703165973',
            'isfamily': 0,
            'isfriend': 0,
            'ispublic': 1,
            'owner': '42881247@N04',
            'secret': '87da0619f7',
            'server': '65535',
            'title': 'Kitten adoption day 220',
            'url_c': 'https://live.staticflickr.com/65535/52703165973_87da0619f7_c.jpg',
            'url_l': 'https://live.staticflickr.com/65535/52703165973_87da0619f7_b.jpg',
            'url_m': 'https://live.staticflickr.com/65535/52703165973_87da0619f7.jpg',
            'url_n': 'https://live.staticflickr.com/65535/52703165973_87da0619f7_n.jpg',
            'url_o': '